# Sales Data Preprocessing & EDA

This notebook demonstrates data import, cleaning, missing value handling, feature selection, transformations, and initial visualizations for a synthetic **Sales** dataset with two related tables: **orders** and **products**.

In [ ]:

import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

# Settings
pd.set_option('display.max_columns', None)
base_dir = "/mnt/data/sales_data_preprocessing_project"
data_dir = os.path.join(base_dir, "data")
fig_dir = os.path.join(base_dir, "figures")
os.makedirs(fig_dir, exist_ok=True)

orders_path = os.path.join(data_dir, "orders.csv")
products_path = os.path.join(data_dir, "products.csv")

orders = pd.read_csv(orders_path)
products = pd.read_csv(products_path)
orders.head()


In [ ]:

print("Orders shape:", orders.shape)
print("Products shape:", products.shape)
print("\nOrders info:")
print(orders.info())
print("\nProducts info:")
print(products.info())


## Data Type Fixes

In [ ]:

# Convert date columns
for col in ["OrderDate", "ShipDate"]:
    orders[col] = pd.to_datetime(orders[col], errors='coerce')

# Ensure numeric types
num_cols = ["Quantity", "UnitPrice", "Discount", "Sales", "Cost", "Profit"]
orders[num_cols] = orders[num_cols].apply(pd.to_numeric, errors='coerce')

# Category types
for col in ["Region", "Priority"]:
    orders[col] = orders[col].astype("category")

products["Category"] = products["Category"].astype("category")
products["SubCategory"] = products["SubCategory"].astype("category")

orders.dtypes


## Remove Duplicates

In [ ]:

before = orders.shape[0]
orders = orders.drop_duplicates()
after = orders.shape[0]
print(f"Removed {before - after} duplicate rows.")


## Missing Values Handling

In [ ]:

# Summarize missing
missing_summary = orders.isna().mean().sort_values(ascending=False)
print(missing_summary)

# Strategy:
# - ShipDate: impute using OrderDate + median ship delta
# - Discount: fill with 0 (assume no discount)
# - Region: fill with mode

# Impute ShipDate
valid_ship = (orders["ShipDate"] - orders["OrderDate"]).dt.days.dropna()
median_ship_days = int(valid_ship.median())
orders["ShipDate"] = orders["ShipDate"].fillna(orders["OrderDate"] + pd.to_timedelta(median_ship_days, unit='D'))

# Discount
orders["Discount"] = orders["Discount"].fillna(0.0)

# Region
mode_region = orders["Region"].mode().iloc[0]
orders["Region"] = orders["Region"].fillna(mode_region)

orders.isna().sum()


## Feature Engineering & Selection

In [ ]:

# Derived features
orders["OrderYear"] = orders["OrderDate"].dt.year
orders["OrderMonth"] = orders["OrderDate"].dt.to_period("M").astype(str)
orders["AOV"] = orders["Sales"]  # per-order value
orders["MarginPct"] = np.where(orders["Sales"]>0, (orders["Profit"]/orders["Sales"])*100, np.nan)

# Select key columns for modeling/BI
selected_cols = ["OrderID","OrderDate","ShipDate","CustomerID","Region","ProductID","Quantity",
                 "UnitPrice","Discount","Sales","Cost","Profit","Priority","OrderYear","OrderMonth","AOV","MarginPct"]
orders_selected = orders[selected_cols].copy()

orders_selected.head()


## Data Integrity Checks

In [ ]:

# 1) Sales should equal Quantity * UnitPrice * (1 - Discount) within rounding tolerance
recalc_sales = (orders_selected["Quantity"] * orders_selected["UnitPrice"] * (1 - orders_selected["Discount"])).round(2)
check_sales = np.isclose(recalc_sales, orders_selected["Sales"], atol=0.01)
print("Sales integrity pass rate:", check_sales.mean())

# 2) Profit = Sales - Cost
recalc_profit = (orders_selected["Sales"] - orders_selected["Cost"]).round(2)
check_profit = np.isclose(recalc_profit, orders_selected["Profit"], atol=0.01)
print("Profit integrity pass rate:", check_profit.mean())


## Join with Products (Data Modeling Preview)

In [ ]:

orders_enriched = orders_selected.merge(products[["ProductID","ProductName","Category","SubCategory"]],
                                        on="ProductID", how="left")
orders_enriched.head()


## Summary Statistics

In [ ]:

summary_numeric = orders_enriched[["Quantity","UnitPrice","Discount","Sales","Cost","Profit","AOV","MarginPct"]].describe().T
summary_categorical = orders_enriched[["Region","Priority","Category","SubCategory"]].astype(str).describe().T
print(summary_numeric)
print("\n")
print(summary_categorical)


## Visualizations

In [ ]:

# 1) Sales by Region (Bar)
region_sales = orders_enriched.groupby("Region")["Sales"].sum().sort_values(ascending=False)
plt.figure()
region_sales.plot(kind="bar", title="Total Sales by Region")
plt.ylabel("Sales")
plt.tight_layout()
plt.savefig(os.path.join(fig_dir, "sales_by_region.png"), dpi=120)
plt.show()


In [ ]:

# 2) Sales Trend by Month (Line)
monthly_sales = orders_enriched.groupby("OrderMonth")["Sales"].sum().sort_index()
plt.figure()
monthly_sales.plot(kind="line", title="Sales Trend by Month")
plt.ylabel("Sales")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(fig_dir, "sales_trend_by_month.png"), dpi=120)
plt.show()


In [ ]:

# 3) Category Distribution (Pie)
cat_sales = orders_enriched.groupby("Category")["Sales"].sum()
plt.figure()
cat_sales.plot(kind="pie", autopct="%1.1f%%", title="Sales Share by Category")
plt.ylabel("")
plt.tight_layout()
plt.savefig(os.path.join(fig_dir, "sales_share_by_category.png"), dpi=120)
plt.show()


In [ ]:

# 4) Profit by SubCategory (Top 10) - Bar
sub_profit = orders_enriched.groupby("SubCategory")["Profit"].sum().sort_values(ascending=False).head(10)
plt.figure()
sub_profit.plot(kind="bar", title="Top 10 SubCategories by Profit")
plt.ylabel("Profit")
plt.tight_layout()
plt.savefig(os.path.join(fig_dir, "top10_subcategories_profit.png"), dpi=120)
plt.show()


In [ ]:

# 5) Scatter: Sales vs Profit
plt.figure()
plt.scatter(orders_enriched["Sales"], orders_enriched["Profit"])
plt.title("Sales vs Profit")
plt.xlabel("Sales")
plt.ylabel("Profit")
plt.tight_layout()
plt.savefig(os.path.join(fig_dir, "sales_vs_profit_scatter.png"), dpi=120)
plt.show()


## Save Cleaned & Enriched Data

In [ ]:

clean_path = os.path.join(data_dir, "orders_enriched_clean.csv")
orders_enriched.to_csv(clean_path, index=False)
clean_path
